In [ ]:
import sys
import os
from src import app
from gevent import monkey,pywsgi
CURRENT_DIR = os.getcwd()
sys.path.insert(0, CURRENT_DIR)

In [ ]:
monkey.patch_all()
server = pywsgi.WSGIServer(('127.0.0.2', 5000), app)
#app.debug = True
server.serve_forever()
#http://127.0.0.2:5000/

The history saving thread hit an unexpected error (LoopExit('This operation would block forever', <Hub at 0x272a0b3e2a8 select pending=0 ref=0>)).History will not be written to the database.


127.0.0.1 - - [2020-02-16 22:38:35] "GET /bonds HTTP/1.1" 200 6970 0.023890
127.0.0.1 - - [2020-02-16 22:38:35] "GET /static/vendor/fontawesome-free/css/all.min.css HTTP/1.1" 200 57493 0.002011
127.0.0.1 - - [2020-02-16 22:38:35] "GET /static/css/template.css HTTP/1.1" 200 16353 0.002004
127.0.0.1 - - [2020-02-16 22:38:35] "GET /static/css/mycss.css HTTP/1.1" 200 1784 0.002004
127.0.0.1 - - [2020-02-16 22:38:35] "GET /static/css/daterangepicker.css HTTP/1.1" 200 8374 0.001988
127.0.0.1 - - [2020-02-16 22:38:35] "GET /static/vendor/jquery/jquery-3.4.1.slim.min.js HTTP/1.1" 200 88468 0.003008
127.0.0.1 - - [2020-02-16 22:38:35] "GET /static/vendor/jquery/popper.min.js HTTP/1.1" 200 21584 0.002001
127.0.0.1 - - [2020-02-16 22:38:35] "GET /static/js/bonds.js HTTP/1.1" 200 1131 0.003007
127.0.0.1 - - [2020-02-16 22:38:35] "GET /static/vendor/daterangepicker/moment.min.js HTTP/1.1" 200 51788 0.002509
127.0.0.1 - - [2020-02-16 22:38:35] "GET /static/vendor/daterangepicker/daterangepicker.min.

In [ ]:
import sys
sys.path.insert(1, r'C:\Users\Kart\jupyter_projects')
import mygit as mg

In [ ]:
folder=r'C:\Users\Kart\jupyter_projects\webtrade'
git_url='git@github.com:Rayveni/web.git'
gt=mg.gitwrapper(folder)
gt.push_commit(git_url,commit_msg='update')

In [ ]:
from jobs import job_sl_bonds
from db_drivers import mongo_manager
cfg={"driver": "mongo", "db_name": "trade", "host": "localhost",
     "port": 27017, "user": "", "user_pswd": "", "mongo_data": "C:\\Users\\DB"}

a=job_sl_bonds(cfg)

In [ ]:
from sys import path
path.append(r"C:\Users\Kart\jupyter_projects\webtrade\webtrade\src")

In [ ]:
from commons import convert_file_size,init_db_manager
from jobs import *
#a=job_sectors(init_db_manager())

In [ ]:
    err,db_stats=__db_stats()
    if not err[0]:
        flash_complex_result(err,db_stats)
    else:
        pass